In [249]:
import pandas as pd
import math
import csv
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.functions 
from pyspark.sql import Row

# read HW2 data by using spark context
#textFile = sc.textFile("file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/News_Final.csv")

# with open ("/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/News_Final.csv",'r',encoding = 'utf8') as file:
#     data = csv.reader(file,delimiter = ",")
#     dataset = list(data)
# print("dataset long:",len(dataset))

#scDataSet = sc.parallelize(dataset)




In [325]:
# Q3 #
topic = ["obama","economy","microsoft","palestine"]


#rdd = pd.read_csv('file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/News_Final.csv')

# input data by using sqlContext, split data by comma, and use header as dataFrame column name
df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true', sep=',').load("file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/News_Final.csv")

# retrivel four topic
for i in range(4):
    print("\t******",topic[i],"******","\n")
    # calcu
    totalScore = df.filter(df['Topic']==topic[i]).select([sum('SentimentTitle')]).collect()
    topicCount = df.filter(df['Topic']==topic[i]).count()
    print(topic[i]+" TotalScore :",totalScore[0][0])
    print(topic[i]+" AveScore :",totalScore[0][0]/topicCount,"\n")
    print("\t******",topic[i],"******","\n\n")
    
# Q3 #

    




	****** obama ****** 

obama TotalScore : -15.74368631545492
obama AveScore : -0.0005814842591119084 

	****** obama ****** 


	****** economy ****** 

economy TotalScore : -336.9370044373353
economy AveScore : -0.010475919672833234 

	****** economy ****** 


	****** microsoft ****** 

microsoft TotalScore : 49.43849052234959
microsoft AveScore : 0.002310425765134573 

	****** microsoft ****** 


	****** palestine ****** 

palestine TotalScore : -164.48440896913712
palestine AveScore : -0.019865266783712213 

	****** palestine ****** 




In [345]:
# Q2 Done#

Facebook = ["Facebook_Economy","Facebook_Microsoft","Facebook_Obama","Facebook_Palestine"]
Google = ["GooglePlus_Economy","GooglePlus_Microsoft","GooglePlus_Palestine"]
Linked = ["LinkedIn_Economy","LinkedIn_Microsoft","LinkedIn_Obama","LinkedIn_Palestine"]

# processing Q2 request
def process(file):
    for f in file:
        # input data from csv by using spark context
        rdd = sc.textFile("file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/"+f+".csv")
        # remove header
        header = rdd.first()
        hrdd = rdd.filter(lambda x:x!= header)
        
        # retrivel all dataset, each time calculate one row
        for i in hrdd.take(rdd.count()):
            ID =0  # for remove IDLink
            perDaySum = 0  # for sum score
            # split row data by split comma
            for j in i.split(","):
                if ID !=0:
                    perDaySum +=float(j)
                else:
                    ID = float(j)
            perDaySum = perDaySum - ID
            #print("AveByHour :",perDaySum/48)
            #print("AveByDay :",perDaySum/2)
        print("Finish")
       
        
#process(Facebook)
#process(Google)
#process(Linked)


# Q2 #

file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/Facebook_Economy.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/Facebook_Microsoft.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/Facebook_Obama.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/Facebook_Palestine.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/LinkedIn_Economy.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/LinkedIn_Microsoft.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/LinkedIn_Obama.csv
Finish
file:/home/ethan/pythonwork/ipynotebook/HW2/HW2Data/LinkedIn_Palestine.csv
Finish
